In [36]:
import requests
import numpy as np
from pprint import pprint

In [37]:
url = "http://127.0.0.1:8800"

In [38]:
response = requests.get(f"{url}/env")
pprint(response.json())

{'objects': {'object_bowl_0': {'id': 209,
                               'ori': [4.152897642893782e-07,
                                       1.576816850867857e-06,
                                       -1.875104609769978e-07],
                               'pos': [-2.1999999758043627,
                                       -0.049999943856495345,
                                       0.9624647727021073]},
             'object_fork_0': {'id': 208,
                               'ori': [6.792080370426803e-17,
                                       0.0,
                                       2.4119956494167827e-18],
                               'pos': [-0.2999999999988103,
                                       -0.6499999999995241,
                                       0.9397844891595659]}},
 'robot': {'arm_joints': [-0.01191072205172512,
                          -1.032572157833481,
                          0.0508923408920614,
                          -2.2597982693466316,
      

In [39]:
response = requests.get(f"{url}/env_entire")
all = response.json()

print(all.keys())

dict_keys(['timestamp', 'objects', 'robot', 'fixtures', 'groups', 'objects_by_group', 'ungrouped_objects'])


In [40]:
# pprint(all["objects"])

# pprint(all["robot"]

# pprint(all["fixtures"])

pprint(all["objects_by_group"])

{'cab_1_left_group': [],
 'cab_2_left_group': [],
 'cab_corner_3_left_group': [],
 'cab_corner_4_left_group': [],
 'cab_main_main_group': [],
 'counter_1_left_left_group': ['object_bowl_0'],
 'counter_1_right_main_group': [],
 'counter_corner_left_group': [],
 'counter_corner_main_main_group': [],
 'counter_main_main_group': [],
 'fridge_cab_main_group': [],
 'fridge_housing_main_group': [],
 'fridge_main_group': [],
 'hood_main_group': [],
 'island_left_group': ['object_fork_0'],
 'microwave_left_group': [],
 'shelves_main_group': [],
 'sink_left_group': [],
 'stack_1_left_group': [],
 'stack_1_main_group': [],
 'stack_2_left_group': [],
 'stack_2_main_group': [],
 'stack_3_left_group': [],
 'stack_3_main_group': [],
 'stovetop_main_group': [],
 'window_group': []}


In [ ]:
pprint(all["ungrouped_objects"])

[]


In [52]:
def make_object_text_dict(url, object_name=None):
    response = requests.get(f"{url}/env_entire")
    all = response.json()

    objects_by_group = all["objects_by_group"]
    ungrouped_objects = all["ungrouped_objects"]

    object_text_dict = {}
    for group_name, objects_list in objects_by_group.items():
        if objects_list:
            print(f"Group: {group_name}")
            for obj in objects_list:
                if object_name is not None:
                    if obj != object_name:
                        continue
                object_text = f"object {obj} is in group {group_name}"
                object_text_dict[obj] = object_text

    if ungrouped_objects:
        raise ValueError("There are ungrouped objects in the environment.")

    return object_text_dict


object_text_dict = make_object_text_dict(url)

pprint(object_text_dict)

Group: counter_1_left_left_group
Group: island_left_group
{'object_bowl_0': 'object object_bowl_0 is in group counter_1_left_left_group',
 'object_fork_0': 'object object_fork_0 is in group island_left_group'}
